In [93]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [94]:
!pip install emoji

In [141]:
import pandas as pd
import numpy as np
import re
import sys
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import csv
import datetime
from bs4 import BeautifulSoup
import itertools
import emoji# emoticons

from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
from sklearn.metrics import classification_report

lemmatizer = WordNetLemmatizer()

def lemmatization(tweet):
  tokens = word_tokenize(tweet)
  tweet = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
  return tweet


def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
  
      ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

# self defined contractions
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
import re
import string
from bs4 import BeautifulSoup

#for tweet in tweet_data:
def preprocess_tweet(tweet):
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
   
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    
    #Removal of hastags/account
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\<\>\:\;\-\=]", " ", tweet).split())
    
    #Lower case
    tweet = tweet.lower()
    
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    
    #Deal with emoticons source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    
    #Deal with emojis
    tweet = emoji.demojize(tweet)

    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())

    tweet = lemmatization(tweet)

    lmtzr = WordNetLemmatizer()
    tweet_list = []
    tweet = tweet.split()
    for tweet_word in tweet:
      tweet_list.append(lmtzr.lemmatize(tweet_word))

    tweet = " "
    tweet = tweet.join(tweet_list)


    return tweet

In [155]:
usecols = (4,6)
dataframe = pd.read_table("/content/task3_training.tsv",usecols=usecols)
dataframe

,extraction,tweet
0,NaN,@seefisch:oral drugs for pyelonephritis:ciprof...
1,NaN,happy for wellbutrin; has similar effects as a...
2,NaN,@stilgarg i'm ok ty have an official diagnosis...
3,NaN,i'm soo depressed cymbalta couldn't help me .
4,NaN,time for my daily afternoon relaxation ritual ...
...,...,...
2241,NaN,i will admit that most people would agree zypr...
2242,gain so much weight,that zyprexa really makes your vocal chords ma...
2243,NaN,@incubator04 i'm hoping that i get on the zypr...
2244,increase my weight,"i'm so fine today. increasing zyprexa,my condi..."


In [154]:
x = "we aren't"
preprocess_tweet(x)

['we', 'are', 'not']


'we are not'

In [0]:
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

In [156]:
train_data=[]
for tweet in dataframe.iterrows():
  tweet_txt = preprocess_tweet(tweet[1][1])
  data = []
  text = []
  entities = []
  extra_txt_s = []
  text = tweet_txt.split()
  if tweet_txt == "":
    continue
  if tweet_txt == " ":
    continue

  if not pd.isnull(tweet[1][0]):
    extra_txt_s =   preprocess_tweet(tweet[1][0])
  
  for word in text:
    if word in extra_txt_s:
      entities.append('ADR')
    else:
      entities.append('O')
  
  data.append(text)
  data.append(entities)

  train_data.append(tuple(data))

['oral', 'drug', 'for', 'pyelonephritis', 'ciprofloxacin', 'levofloxacin', 'tmp/smz', 'do', 'not', 'use', 'nitrofurantoin', 'for', 'pyelo(only', 'cystitis)', '_medinaf']
['happy', 'for', 'wellbutrin', 'ha', 'similar', 'effect', 'a', 'adderall', 'trazodone', 'is', 'super', 'promising', 'for', 'sleep', 'but', 'abilify', 'can', 'cause', 'weight', 'gain', '_']
["i'm", 'ok', 'ty', 'have', 'an', 'official', 'diagnosis', 'of', 'bipolar', 'now', 'feeling', 'ok', 'at', 'the', 'moment', 'lamotrigine', 'ha', 'been', 'increased', 'having', 'monotherapy', '/']
["i'm", 'soo', 'depressed', 'cymbalta', 'could', 'not', 'help', 'me']
['time', 'for', 'my', 'daily', 'afternoon', 'relaxation', 'ritual', 'of', 'smoking', 'weed', 'taking', '2', 'mg', 'of', 'clonazepam', 'and', '400', 'mg', 'of', 'seroquel', 'xr']
['questionable', 'x', 'ray', 'result', 'nothing', 'a', 'little', 'levaquin', 'will', 'not', 'help', 'oh', 'yeah', 'and', 'the', 'cough', 'med', 'with', 'codeine']
['doe', 'nicotine', 'gum', 'actuall

In [0]:
usecols = (4,6)
dataframe = pd.read_table("/content/task3_validation.tsv",usecols=usecols)

In [158]:
eval_data=[]
for tweet in dataframe.iterrows():
  tweet_txt = preprocess_tweet(tweet[1][1])
  data = []
  text = []
  entities = []
  extra_txt_s = []
  text = tweet_txt.split()
  
  if not pd.isnull(tweet[1][0]):
    extra_txt_s =   preprocess_tweet(tweet[1][0])
  
  for word in text:
    if word in extra_txt_s:
      entities.append('ADR')
    else:
      entities.append('O')
  
  data.append(text)
  data.append(entities)

  eval_data.append(tuple(data))

['do', 'you', 'have', 'any', 'medication', 'allergy', '"asthma', '"', 'me', '"', '"', 'pt', '"no', 'wait', 'avelox', 'that', 'is', 'it', '"', '"so', 'no', 'other', 'allergy', '"', '"right', '"', '*cont']
['allergy']
['if', 'ha', 'hurt', 'your', 'liver', 'avoid', 'tylenol', 'always', 'a', 'it', 'further', 'damage', 'liver', 'eat', 'grapefruit', 'unless', 'taking', 'cardiac', 'drug']
['hurt', 'your', 'liver']
['apparently', 'baclofen', 'greatly', 'exacerbates', 'the', '"ad"', 'part', 'of', 'my', 'adhd', 'average', 'length', 'of', 'focus', 'today', 'about', '30', 'second']
['ad']
['apparently', 'baclofen', 'greatly', 'exacerbates', 'the', '"ad"', 'part', 'of', 'my', 'adhd', 'average', 'length', 'of', 'focus', 'today', 'about', '30', 'second']
['focus']
['pt', 'of', 'mine', 'died', 'from', 'cipro', 'rt', 'if', 'only', 'more', 'doctor', 'thought', 'like', 'you', 'i', 'lost', 'my', 'entire', 'life', 'to', '12', 'cipro', 'pill']
['died']
['the', 'only', 'pt', 'of', 'mine', 'who', 'ever', 'die

# Now Starts BiLSTM- CRF Model **Pytorch**

In [159]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [0]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [0]:

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq



In [162]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
# training_data = [(
#     "the wall street journal reported today that apple corporation made money".split(),
#     "B I I I O O O B I O O".split()
# ), (
#     "georgia tech is a university in georgia".split(),
#     "B I O O O O B".split()
# )]

training_data = train_data

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"O": 0, "ADR": 1, START_TAG: 2, STOP_TAG: 3}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        100):  # again, normally you would NOT do 300 epochs, it is toy data
    print(epoch)
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))

(tensor(33.8418), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
(tensor(34.8140), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [163]:
usecols = (4,6)
dataframe = pd.read_table("/content/task3_validation.tsv",usecols=usecols)

eval_data=[]
y_test=[]
y_pred=[]
count = 0
for tweet in dataframe.iterrows():
  tweet_txt = preprocess_tweet(tweet[1][1])
  data = []
  text = []
  entities = []
  extra_txt_s = []
  text = tweet_txt.split()
  breakTweet = False
  while(True):
    try:
      with torch.no_grad():
    
        if text == "":
          break
        if text == " ":
          break 
        precheck_sent = prepare_sequence(text, word_to_ix)
        y_pred = y_pred + model(precheck_sent)[1]
        break
    except KeyError as e:
      # print(text[0])
      # print(type(str(e)))
      # print(str(e).replace("'",""))
      try:
        text.remove(preprocess_tweet( str(e)))
      except:
        count = count+1
        breakTweet = True
        break
      continue


  if breakTweet:
      continue
  
  if not pd.isnull(tweet[1][0]):
    extra_txt_s =   preprocess_tweet(tweet[1][0])
  for word in text:
    if word in extra_txt_s:
      entities.append('ADR')
      y_test.append(1)
    else:
      entities.append('O')
      y_test.append(0)
  
  data.append(text)
  data.append(entities)

  eval_data.append(tuple(data))

['do', 'you', 'have', 'any', 'medication', 'allergy', '"asthma', '"', 'me', '"', '"', 'pt', '"no', 'wait', 'avelox', 'that', 'is', 'it', '"', '"so', 'no', 'other', 'allergy', '"', '"right', '"', '*cont']
['\'"asthma\'']
['if', 'ha', 'hurt', 'your', 'liver', 'avoid', 'tylenol', 'always', 'a', 'it', 'further', 'damage', 'liver', 'eat', 'grapefruit', 'unless', 'taking', 'cardiac', 'drug']
["'avoid'"]
['apparently', 'baclofen', 'greatly', 'exacerbates', 'the', '"ad"', 'part', 'of', 'my', 'adhd', 'average', 'length', 'of', 'focus', 'today', 'about', '30', 'second']
["'greatly'"]
['apparently', 'baclofen', 'greatly', 'exacerbates', 'the', '"ad"', 'part', 'of', 'my', 'adhd', 'average', 'length', 'of', 'focus', 'today', 'about', '30', 'second']
["'greatly'"]
['pt', 'of', 'mine', 'died', 'from', 'cipro', 'rt', 'if', 'only', 'more', 'doctor', 'thought', 'like', 'you', 'i', 'lost', 'my', 'entire', 'life', 'to', '12', 'cipro', 'pill']
['died']
['the', 'only', 'pt', 'of', 'mine', 'who', 'ever', 'di

In [164]:
print(len(y_pred))
print(len(y_test))

2297
2297


In [165]:

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      1973
           1       0.43      0.24      0.31       324

    accuracy                           0.85      2297
   macro avg       0.66      0.60      0.61      2297
weighted avg       0.82      0.85      0.83      2297



In [0]:
while i < len(dataframe):
  extraction = preprocess_tweet(str(dataframe.loc[i][0]))
  tweet = preprocess_tweet(dataframe.loc[i][1])

  try:
    with torch.no_grad():
      precheck_sent = prepare_sequence(text, word_to_ix)
      y_pred = y_pred + model(precheck_sent)[1]
  except Exception as e:
    count = count+1
    print(e)
    
    break

    
  
  i = i+1






In [0]:
len(dataframe)

In [0]:
print(dataframe.loc[1][1])